In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import os
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import xgboost as xgb
from scipy.spatial.distance import squareform, pdist
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import DistanceMetric
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.mixture import GaussianMixture
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from tqdm import tqdm
import random

# 전처리

## 데이터 전처리

### Round2의 weather 예측값 전처리(Expert model의 input)

In [ ]:
weather_real = pd.read_csv("./weather_actual.csv")
weather_index = weather_real['time']

weather = pd.read_csv("./weather_forecast.csv")
weather1 = weather[weather['round']==1].copy()
weather2 = weather[weather['round']==2].copy()
weather1_index = weather1.reset_index(drop=True)['time']
weather2_index = weather2.reset_index(drop=True)['time']
weather1_val = pd.read_csv("./weather_forecast_round1_val.csv")
weather2_val = pd.read_csv("./weather_forecast_round2_val.csv")

pred = pd.read_csv("./pred.csv")
pred1 = pred[pred['round']==1].copy()
pred2 = pred[pred['round']==2].copy()
pred1_index = pred1.reset_index(drop=True)['time']
pred2_index = pred2.reset_index(drop=True)['time']
pred1_val = pd.read_csv("./gen_forecast_round1_val.csv")
pred2_val = pd.read_csv("./gen_forecast_round2_val.csv")

gen = pd.read_csv("./gens.csv")
gen_val = pd.read_csv("./add_gens.csv")
gen_index = gen['time']

In [ ]:
A = np.array(weather2_index.tolist())
B = np.array(pred2_index.tolist())
C = np.array(gen_index.tolist())
A1 = np.array(weather1_index.tolist())
B1 = np.array(pred1_index.tolist())
total_index = np.intersect1d(np.intersect1d(A,B),C)

In [ ]:
for i in list(np.setdiff1d(A,total_index)):
    weather2.drop([weather2[weather2['time']==i].index.item()],inplace=True)

for i in list(np.setdiff1d(B,total_index)):
    pred2.drop(pred2[pred2['time']==i].transpose().columns,inplace=True)

for i in list(np.setdiff1d(A1,total_index)):
    weather1.drop([weather1[weather1['time']==i].index.item()],inplace=True)

for i in list(np.setdiff1d(B1,total_index)):
    pred1.drop(pred1[pred1['time']==i].transpose().columns,inplace=True)

for i in list(np.setdiff1d(C,total_index)):
    gen.drop([gen[gen['time']==i].index.item()],inplace=True)


In [ ]:
# real_weather & pred_weather 오차
weather1 = weather1.drop(['round','time'],axis=1)
weather2 = weather2.drop(['round','time'],axis=1)
weather1_val = weather1_val.drop(['round','time'],axis=1)
weather2_val = weather2_val.drop(['round','time'],axis=1)

ss1 = MinMaxScaler()
ss2 = MinMaxScaler()

ss_weather1 = pd.DataFrame(ss1.fit_transform(weather1))
ss_weather2 = pd.DataFrame(ss2.fit_transform(weather2))
ss_weather1_val = pd.DataFrame(ss1.transform(weather1_val))
ss_weather2_val = pd.DataFrame(ss2.transform(weather2_val))
ss_weather1, ss_weather2

# ss_weather = pd.concat([ss_weather, (weather-weather_real)/(weather_real.min()-weather_real.max())],axis=1)

### Round2의 Expert model 결과값 전처리

In [ ]:
pred1 = pred1.drop(['round', 'time'],axis=1)
pred2 = pred2.drop(['round', 'time'],axis=1)
pred1_val = pred1_val.drop(['round', 'time'],axis=1)
pred2_val = pred2_val.drop(['round', 'time'],axis=1)
for i in range(5):
    exec(f"pred{i}_1 = pred1[pred1['model_id']==i].drop(['model_id'], axis=1).reset_index(drop=True)")
    exec(f"pred{i}_1.columns = ['model{i}']")
    exec(f"pred{i}_2 = pred2[pred2['model_id']==i].drop(['model_id'], axis=1).reset_index(drop=True)")
    exec(f"pred{i}_2.columns = ['model{i}']")

    exec(f"pred{i}_1_val = pred1_val[pred1_val['model_id']==i].drop(['model_id'], axis=1).reset_index(drop=True)")
    exec(f"pred{i}_1_val.columns = ['model{i}']")
    exec(f"pred{i}_2_val = pred2_val[pred2_val['model_id']==i].drop(['model_id'], axis=1).reset_index(drop=True)")
    exec(f"pred{i}_2_val.columns = ['model{i}']")
    
gen_pred1 = pd.concat([eval(f'pred{i}_1') for i in range(5)], axis=1)
gen_pred2 = pd.concat([eval(f'pred{i}_2') for i in range(5)], axis=1)
gen_pred1_val = pd.concat([eval(f'pred{i}_1_val') for i in range(5)], axis=1)
gen_pred2_val = pd.concat([eval(f'pred{i}_2_val') for i in range(5)], axis=1)

gen_pred1, gen_pred2, gen_pred1_val, gen_pred2_val

### 실제 gen 값 전처리

In [ ]:
gen = gen.drop(['time'], axis=1)
gen_val = gen_val.drop(['time'], axis=1)
gen, gen_val

## 학습/검증 데이터 전처리

###  Validation Shape는 11월 12일 00:10 기준 24x22 = 528 행이어야 함

In [ ]:
ss_weather_dataset1 = torch.tensor(np.array(ss_weather1)).type(torch.float32)
gen_pred_dataset1 = torch.tensor(np.array(gen_pred1)).type(torch.float32)
ss_weather_dataset2 = torch.tensor(np.array(ss_weather2)).type(torch.float32)
gen_pred_dataset2 = torch.tensor(np.array(gen_pred2)).type(torch.float32)
gen_dataset = torch.tensor(np.array(gen)).type(torch.float32)

ss_weather_dataset1_val = torch.tensor(np.array(ss_weather1_val)).type(torch.float32)
gen_pred_dataset1_val = torch.tensor(np.array(gen_pred1_val)).type(torch.float32)
ss_weather_dataset2_val = torch.tensor(np.array(ss_weather2_val)).type(torch.float32)
gen_pred_dataset2_val = torch.tensor(np.array(gen_pred2_val)).type(torch.float32)
gen_dataset_val = torch.tensor(np.array(gen_val)).type(torch.float32)

ss_weather_dataset1.shape, gen_pred_dataset1.shape, ss_weather_dataset2.shape, gen_pred_dataset2.shape, gen_dataset.shape, ss_weather_dataset1_val.shape, gen_pred_dataset1_val.shape, ss_weather_dataset2_val.shape, gen_pred_dataset2_val.shape, gen_dataset_val.shape

In [ ]:
# weather_pred 13 & gen_pred 5 & gen 1 
total_dataset1 = torch.cat([ss_weather_dataset1, gen_pred_dataset1, gen_dataset], dim=1)
total_dataset2 = torch.cat([ss_weather_dataset2, gen_pred_dataset2, gen_dataset], dim=1)
total_dataset1_val = torch.cat([ss_weather_dataset1_val, gen_pred_dataset1_val, gen_dataset_val], dim=1)
total_dataset2_val = torch.cat([ss_weather_dataset2_val, gen_pred_dataset2_val, gen_dataset_val], dim=1)

total_dataset1.shape, total_dataset2.shape, total_dataset1_val.shape, total_dataset2_val.shape,      

In [ ]:
# 시간 고려 -> Train : Val = 앞 90% : 뒤 10%
train_dataset1 = total_dataset1
val_dataset1 = total_dataset1_val
train_dataset2 = total_dataset2
val_dataset2 = total_dataset2_val

train_dataset1.shape, val_dataset1.shape, train_dataset2.shape, val_dataset2.shape

In [ ]:
# batch dataset(input_X : pred_weather, infer_X : pred_gen, true_Y : gen)
def make_batch_dataset(train_dataset, random_idx):
    batch_dataset = train_dataset[random_idx]
    batch_input_X = batch_dataset[:,:18]
    batch_infer_X = batch_dataset[:,13:18]
    batch_true_Y = batch_dataset[:,[18]]
    return batch_input_X, batch_infer_X, batch_true_Y

In [ ]:
def make_validation_dataset(val_dataset):
    batch_dataset = val_dataset
    batch_input_X = batch_dataset[:,:18]
    batch_infer_X = batch_dataset[:,13:18]
    batch_true_Y = batch_dataset[:,[18]]
    return batch_input_X, batch_infer_X, batch_true_Y

# Loss Function

In [ ]:
def train_loss10(batch_true_Y, batch_pred_Y1):
    # Utilization preprocess
    batch_true_Y[batch_true_Y < 9.9] = 0

    error_abs = torch.abs(batch_pred_Y1- batch_true_Y)
    error_type1 = torch.sqrt(error_abs[((error_abs/99*100)>6) & ((error_abs/99*100)<=8) & (batch_true_Y > 0)])
    error_type2 = torch.sqrt(error_abs[((error_abs/99*100)>8) & (batch_true_Y > 0)])
    cost_type1 = batch_true_Y[((error_abs/99*100)>6) & ((error_abs/99*100)<=8) & (batch_true_Y > 0)]
    cost_type2 = batch_true_Y[((error_abs/99*100)>8) & (batch_true_Y > 0)] * 4
    error_type1_num = len(cost_type1)
    error_type2_num = len(cost_type2)
    batch_size = len(batch_true_Y[batch_true_Y > 0])

    # loss
    if (error_type1_num != 0) and (error_type2_num != 0):
        loss = (torch.sum(error_type1 * cost_type1) + torch.sum(error_type2 * cost_type2)) / (batch_size)
    elif (error_type1_num != 0):
        loss = torch.sum(error_type1 * cost_type1) / (batch_size)
    elif (error_type2_num != 0):
        loss = torch.sum(error_type2 * cost_type2) / (batch_size)
    else:
        loss = torch.tensor(0.)

    return loss

In [ ]:
def validation10(val_batch_true_Y, val_batch_pred_Y1):
    val_true_Y = np.array(val_batch_true_Y.detach())
    val_pred_Y1 = np.array(val_batch_pred_Y1.detach())

    val_true_Y[val_true_Y < 9.9] = 0

    error_abs = np.abs(val_pred_Y1 - val_true_Y)
    cost_type1 = val_true_Y[((error_abs/99*100)>6) & ((error_abs/99*100)<=8) & (val_true_Y > 0)]
    cost_type2 = val_true_Y[((error_abs/99*100)>8) & (val_true_Y > 0)] * 4
    full_incentive = np.sum(val_true_Y[(val_true_Y > 0)] * 4)
    error_type1_num = len(cost_type1)
    error_type2_num = len(cost_type2)

    # loss
    if (error_type1_num != 0) and (error_type2_num != 0):
        loss = (np.sum(cost_type1) + np.sum(cost_type2))
    elif (error_type1_num != 0):
        loss = np.sum(cost_type1)
    elif (error_type2_num != 0):
        loss = np.sum(cost_type2)
    else:
        loss = 0.

    
    return loss, (1-loss/full_incentive)*100

In [ ]:
def validation(val_batch_true_Y, val_batch_pred_Y1, val_batch_pred_Y2):
    val_true_Y = np.array(val_batch_true_Y.detach())
    val_pred_Y1 = np.array(val_batch_pred_Y1.detach())
    val_pred_Y2 = np.array(val_batch_pred_Y2.detach())

    val_true_Y[val_true_Y < 9.9] = 0

    error_abs = np.abs(np.round((val_pred_Y1+val_pred_Y2)/2) - val_true_Y)
    cost_type1 = val_true_Y[((error_abs/99*100)>6) & ((error_abs/99*100)<=8) & (val_true_Y > 0)]
    cost_type2 = val_true_Y[((error_abs/99*100)>8) & (val_true_Y > 0)] * 4
    full_incentive = np.sum(val_true_Y[(val_true_Y > 0)] * 4)
    error_type1_num = len(cost_type1)
    error_type2_num = len(cost_type2)

    # loss
    if (error_type1_num != 0) and (error_type2_num != 0):
        loss = (np.sum(cost_type1) + np.sum(cost_type2))
    elif (error_type1_num != 0):
        loss = np.sum(cost_type1)
    elif (error_type2_num != 0):
        loss = np.sum(cost_type2)
    else:
        loss = 0.

    
    return loss, (1-loss/full_incentive)*100, full_incentive-loss

# Model Architecture

In [ ]:
class Ensemble_model1(nn.Module):

    def __init__(self):
        super(Ensemble_model1, self).__init__()
        self.fe = nn.Linear(18, 128)
        self.hidden1 = nn.Linear(128,128)
        self.hidden2 = nn.Linear(128,128)
        self.hidden3 = nn.Linear(128,128)
        self.hidden4 = nn.Linear(128,128)
        self.latent = nn.Linear(128, 5)
        self.relu = nn.ReLU()

    def forward(self, x, x1):
        batch_size = len(x)
        x = self.fe(x)
        x = self.relu(x)
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.hidden3(x)
        x = self.relu(x)
        x = self.hidden4(x)
        x = self.relu(x)
        f = self.latent(x)
        x = torch.sum(f*x1, 1).reshape(batch_size, -1)
        x = self.relu(x)
        return (x, f)

In [ ]:
class Ensemble_model2(nn.Module):

    def __init__(self):
        super(Ensemble_model2, self).__init__()
        self.fe = nn.Linear(18, 64)
        self.hidden1 = nn.Linear(64,64)
        self.hidden2 = nn.Linear(64,64)
        self.hidden3 = nn.Linear(64,64)
        self.hidden4 = nn.Linear(64,64)
        self.hidden5 = nn.Linear(64,64)
        self.latent = nn.Linear(64, 5)
        self.relu = nn.ReLU()

    def forward(self, x, x1):
        batch_size = len(x)
        x = self.fe(x)
        x = self.relu(x)
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.hidden3(x)
        x = self.relu(x)
        x = self.hidden4(x)
        x = self.relu(x)
        x = self.hidden5(x)
        x = self.relu(x)
        f = self.latent(x)
        x = torch.sum(f*x1, 1).reshape(batch_size, -1)
        x = self.relu(x)
        return (x, f)

### 아래의 train_date는 입찰날짜+1 or 예측 대상 날짜를 입력하면 됨
#### 11월 12일 00:10 기준 2023-11-13 임!

In [ ]:
####################### 반드시 train_date 수정할 것!!!!!!!! ##########################
train_date = "2023-11-13"
####################### 반드시 train_date 수정할 것!!!!!!!! ##########################


model1 = Ensemble_model1()
model2 = Ensemble_model2()

import copy
epoch = 1000000
lr = 1e-5
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
batch_size1 = 128
batch_size2 = 128
best_val_loss = 10000000000
best_accuracy = 0
best_model1 = model1
best_model2 = model2
local_best_model1 = model1
local_best_model2 = model2
local_val_loss1 = 1000000000
local_val_loss2 = 1000000000


val_input_X1, val_infer_X1, val_true_Y = make_validation_dataset(val_dataset1)
val_input_X2, val_infer_X2, val_true_Y = make_validation_dataset(val_dataset2)

for epoch_idx in range(epoch):
    
    random_idx = random.sample(range(len(train_dataset1)), k=batch_size1)
    batch_input_X1, batch_infer_X1, batch_true_Y = make_batch_dataset(train_dataset1, random_idx)
    model1.train()
    optimizer1.zero_grad()
    batch_pred_Y1 = model1(batch_input_X1, batch_infer_X1)[0]
    loss = train_loss10(batch_true_Y, batch_pred_Y1)
    loss.backward()
    optimizer1.step()
    model1.eval()
    
    random_idx = random.sample(range(len(train_dataset1)), k=batch_size2)
    batch_input_X2, batch_infer_X2, batch_true_Y = make_batch_dataset(train_dataset2, random_idx)
    model2.train()
    optimizer2.zero_grad()
    batch_pred_Y2 = model2(batch_input_X2, batch_infer_X2)[0]
    loss = train_loss10(batch_true_Y, batch_pred_Y2)
    loss.backward()
    optimizer2.step()
    model2.eval()

    temp_val1, _ = validation10(val_true_Y, model1(val_input_X1, val_infer_X1)[0])
    temp_val2, _ = validation10(val_true_Y, model2(val_input_X2, val_infer_X2)[0])
    if temp_val1 < local_val_loss1:
        local_val_loss1 = temp_val1
        local_best_model1 = copy.deepcopy(model1)
    
    if temp_val2 < local_val_loss2:
        local_val_loss2 = temp_val2
        local_best_model2 = copy.deepcopy(model2)
    
    local_best_model1.eval()
    local_best_model2.eval()
    val_loss, val_accuracy, val_incen = validation(val_true_Y, local_best_model1(val_input_X1, val_infer_X1)[0], model2(val_input_X2, val_infer_X2)[0])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_accuracy = val_accuracy
        _, temp_val1 = validation10(val_true_Y, local_best_model1(val_input_X1, val_infer_X1)[0])
        _, temp_val2 = validation10(val_true_Y, model2(val_input_X2, val_infer_X2)[0])

        print(f"epoch : {epoch_idx+1}, efficiency: {best_accuracy:.2f}%({temp_val1:.2f}||{temp_val2:.2f}), incentive : {val_incen}, loss : {best_val_loss}")
        best_model1 = copy.deepcopy(local_best_model1)
        best_model2 = copy.deepcopy(model2)
        torch.save(best_model1, f"best_model_round1_{train_date}.pt")
        torch.save(best_model2, f"best_model_round2_{train_date}.pt")
    
    val_loss, val_accuracy, val_incen = validation(val_true_Y, model1(val_input_X1, val_infer_X1)[0], local_best_model2(val_input_X2, val_infer_X2)[0])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_accuracy = val_accuracy
        _, temp_val1 = validation10(val_true_Y, model1(val_input_X1, val_infer_X1)[0])
        _, temp_val2 = validation10(val_true_Y, local_best_model2(val_input_X2, val_infer_X2)[0])

        print(f"epoch : {epoch_idx+1}, efficiency: {best_accuracy:.2f}%({temp_val1:.2f}||{temp_val2:.2f}), incentive : {val_incen}, loss : {best_val_loss}")
        best_model1 = copy.deepcopy(model1)
        best_model2 = copy.deepcopy(local_best_model2)
        torch.save(best_model1, f"best_model_round1_{train_date}.pt")
        torch.save(best_model2, f"best_model_round2_{train_date}.pt")
    


    val_loss, val_accuracy, val_incen = validation(val_true_Y, model1(val_input_X1, val_infer_X1)[0], model2(val_input_X2, val_infer_X2)[0])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_accuracy = val_accuracy
        _, temp_val1 = validation10(val_true_Y, model1(val_input_X1, val_infer_X1)[0])
        _, temp_val2 = validation10(val_true_Y, model2(val_input_X2, val_infer_X2)[0])

        print(f"epoch : {epoch_idx+1}, efficiency: {best_accuracy:.2f}%({temp_val1:.2f}||{temp_val2:.2f}), incentive : {val_incen}, loss : {best_val_loss}")
        best_model1 = copy.deepcopy(model1)
        best_model2 = copy.deepcopy(model2)
        torch.save(best_model1, f"best_model_round1_{train_date}.pt")
        torch.save(best_model2, f"best_model_round2_{train_date}.pt")
    else:
        val_loss, val_accuracy, val_incen = validation(val_true_Y, best_model1(val_input_X1, val_infer_X1)[0], model2(val_input_X2, val_infer_X2)[0])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_accuracy = val_accuracy
            _, temp_val1 = validation10(val_true_Y, best_model1(val_input_X1, val_infer_X1)[0])
            _, temp_val2 = validation10(val_true_Y, model2(val_input_X2, val_infer_X2)[0])

            print(f"epoch : {epoch_idx+1}, efficiency: {best_accuracy:.2f}%({temp_val1:.2f}||{temp_val2:.2f}), incentive : {val_incen}, loss : {best_val_loss}")            
            best_model2 = copy.deepcopy(model2)
            torch.save(best_model2, f"best_model_round2_{train_date}.pt")
        else:
            val_loss, val_accuracy, val_incen = validation(val_true_Y, model1(val_input_X1, val_infer_X1)[0], best_model2(val_input_X2, val_infer_X2)[0])
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_accuracy = val_accuracy 
                _, temp_val1 = validation10(val_true_Y, model1(val_input_X1, val_infer_X1))
                _, temp_val2 = validation10(val_true_Y, best_model2(val_input_X2, val_infer_X2))

                print(f"epoch : {epoch_idx+1}, efficiency: {best_accuracy:.2f}%({temp_val1:.2f}||{temp_val2:.2f}), incentive : {val_incen}, loss : {best_val_loss}")
                best_model1 = copy.deepcopy(model1)
                torch.save(best_model1, f"best_model_round1_{train_date}.pt")